# Cleaning Public Support data

## Importing necessary libraries

In [1]:
pip install pandas matplotlib seaborn wordcloud


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime,date, timedelta

## Loading the data

In [3]:
slack = pd.read_csv('../sources/support-channels.csv')


## Discover

In [4]:
print('Shape of slack dataframe before cleaning:', slack.shape)

Shape of slack dataframe before cleaning: (481, 14)


In [5]:
slack.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 481 entries, 0 to 480
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Channel_ID        481 non-null    object
 1   Channel_Slug      481 non-null    object
 2   Timestamp         481 non-null    object
 3   Timestamp_Thread  368 non-null    object
 4   User_ID           481 non-null    object
 5   Full_Name         470 non-null    object
 6   Email             481 non-null    object
 7   Permalink         481 non-null    object
 8   Text              481 non-null    object
 9   Text_raw          481 non-null    object
 10  Slack_username    481 non-null    object
 11  Team_ID           481 non-null    object
 12  Team_Name         481 non-null    object
 13  Is_Bot            481 non-null    bool  
dtypes: bool(1), object(13)
memory usage: 49.4+ KB


In [6]:
slack.head(6)

,Channel_ID,Channel_Slug,Timestamp,Timestamp_Thread,User_ID,Full_Name,Email,Permalink,Text,Text_raw,Slack_username,Team_ID,Team_Name,Is_Bot
0,CAZ9W99U4,public-support-full-stack,11/11/2022 15:23:37,11/11/2022 14:44:11,U0426RW6CR5,Elías Fernández,atukdibe@gmail.com,https://4geeksacademy.slack.com/archives/CAZ9W...,fijate si te lo toma solo con background:,fijate si te lo toma solo con background:,atukdibe,T0BFXMWMV,4Geeks Academy,False
1,CAZ9W99U4,public-support-full-stack,11/9/2022 7:52:50,NaN,U03SWDR1KTM,Jack Caldwell-Nichols,jack.wcn@gmail.com,https://4geeksacademy.slack.com/archives/CAZ9W...,Hola a todos! Socorro! Quiero eliminar los fav...,Hola a todos! Socorro! Quiero eliminar los fav...,jack.wcn,T0BFXMWMV,4Geeks Academy,False
2,CAZ9W99U4,public-support-full-stack,11/11/2022 16:12:26,NaN,U03A7FCVDMW,Richard Jardine,rhjardine@gmail.com,https://4geeksacademy.slack.com/archives/CAZ9W...,"Hello. Please, I require guidance regarding th...","Hello. Please, I require guidance regarding th...",rhjardine,T0BFXMWMV,4Geeks Academy,False
3,CAZ9W99U4,public-support-full-stack,11/8/2022 18:24:50,NaN,U03DP0YAHHV,Juan Repeto,juanrepeto@gmail.com,https://4geeksacademy.slack.com/archives/CAZ9W...,Hi there! I'm trying to deploy my final projec...,Hi there! I'm trying to deploy my final projec...,juanrepeto,T0BFXMWMV,4Geeks Academy,False
4,CAZ9W99U4,public-support-full-stack,11/3/2022 16:41:34,NaN,U03JFEQMCTV,Kengee Andrade cruz,kgthcruz94@gmail.com,https://4geeksacademy.slack.com/archives/CAZ9W...,"Hola, alguien me puede ayudar con este proyect...","Hola, alguien me puede ayudar con este proyect...",kgthcruz94,T0BFXMWMV,4Geeks Academy,False
5,CAZ9W99U4,public-support-full-stack,11/7/2022 16:59:39,NaN,U01SJ480RBR,Maikol Moreira,Niukeitor@hotmail.com,https://4geeksacademy.slack.com/archives/CAZ9W...,Hello,Hello,niukeitor,T0BFXMWMV,4Geeks Academy,False


**Creating 2 new columns**

In [7]:
slack['Is_a_question'] = np.where(slack['Timestamp_Thread'].isnull(), 1, 0)

In [8]:
support_agents = ['1','5301']

slack['Is_agent']= np.where(slack['User_ID'].isin(support_agents),1,0)

**Converting timestamp columns**

In [9]:
slack['Datetime'] = pd.to_datetime(slack['Timestamp'])
slack['Datetime_Thread'] = pd.to_datetime(slack['Timestamp_Thread'])

**Creating 2 dataframes: questions and answers**

In [10]:
questions_df = slack[slack['Is_a_question'] == 1]
answers_df = slack[slack['Is_a_question'] == 0]

**Grouping by id and ordering dates in both dataframes**

In [11]:
questions_df = questions_df.groupby("User_ID",group_keys=True).apply(lambda x: x.sort_values(["Datetime"]))
answers_df = answers_df.groupby("User_ID",group_keys=True).apply(lambda x: x.sort_values(["Datetime"]))

In [12]:
questions_df['Minutes_Since_Previous'] = (questions_df['Datetime'] - questions_df['Datetime'].shift(1)).astype('timedelta64[s]')
answers_df['Minutes_Since_Previous'] = (answers_df['Datetime'] - answers_df['Datetime'].shift(1)).astype('timedelta64[s]')

In [13]:
questions_df["diff"] = questions_df["Datetime"].diff()
answers_df["diff"] = answers_df["Datetime"].diff()


In [24]:
def change_first(df,group,col):

    for row in df.itertuples():
        for g in df['User_ID']:
            df[col]= 0




In [25]:
change_first(answers_df, 'User_ID','Minutes_Since_Previous')

NameError: name 'index' is not defined

In [ ]:
questions_df = questions_df.set_index('User_ID')
answers_df = answers_df.set_index('User_ID')

**Number of interactions per student**

**Number of questions per student**

## Data Cleaning

**Encoding boolean column**

**Cleaning joined dataframe**

In [ ]:
SELECT User_ID
    ,Full_Name
    ,Timestamp
    ,STUFF((SELECT ',' + value FROM table t2 WHERE t2.User_ID = t1.User_ID and t2.Full_Name = t1.Full_Name AND t2.Timestamp = t1.Timestamp FOR XML PATH('')),1,1,'') AS value
FROM table t1
GROUP BY ID,name,prop

SyntaxError: invalid syntax (2363840415.py, line 1)